baseline fastText для классификации стилей


- загрузка данных из `data/taiga_style_dataset.csv`;
- подготовка текстов и меток к формату fastText;
- обучение `fasttext.train_supervised`;
- проверка точности и отчёт по метрикам;
- сохранение модели.


In [27]:
# Устанавливаем fastText при необходимости и подключаем зависимости
import importlib
import subprocess
import sys
from pathlib import Path
from tempfile import NamedTemporaryFile

def ensure_fasttext():
    try:
        return importlib.import_module("fasttext")
    except ModuleNotFoundError:
        candidates = [
            ("fasttext-wheel==0.9.2", []),
            ("fasttext==0.9.2", ["--no-build-isolation", "--use-pep517"])
        ]
        last_err = None
        for package, extra_args in candidates:
            try:
                subprocess.run([sys.executable, "-m", "pip", "install", package, *extra_args], check=True)
                break
            except subprocess.CalledProcessError as err:
                last_err = err
        else:
            raise ModuleNotFoundError("fasttext is not installed and installation attempts failed") from last_err
        return importlib.import_module("fasttext")

fasttext = ensure_fasttext()

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

import random

# NumPy 2.0 compatibility shim for fastText predict()
_FastTextClass = fasttext.FastText._FastText

def _predict_numpy_safe(self, text, k=1, threshold=0.0, on_unicode_error="strict"):
    def check(entry: str) -> str:
        if entry.find("\n") != -1:
            raise ValueError("predict processes one line at a time (remove \"\\n\")")
        return entry + "\n"

    if isinstance(text, list):
        text = [check(entry) for entry in text]
        all_labels, all_probs = self.f.multilinePredict(text, k, threshold, on_unicode_error)
        return all_labels, all_probs

    text = check(text)
    predictions = self.f.predict(text, k, threshold, on_unicode_error)
    if predictions:
        probs, labels = zip(*predictions)
    else:
        probs, labels = ([], ())
    return labels, np.asarray(probs)

_FastTextClass.predict = _predict_numpy_safe



In [28]:
# Конфигурация путей и воспроизводимость
DATA_PATH = Path("data/taiga_style_dataset.csv")
MODEL_DIR = Path("models")
RANDOM_SEED = 42
VAL_SIZE = 0.2

MODEL_DIR.mkdir(parents=True, exist_ok=True)

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)


In [29]:
# Загружаем и минимально очищаем корпус
df = (
    pd.read_csv(DATA_PATH)
    .rename(columns={"style_hint_label": "label_id", "style_hint": "label_name"})
)
df["text"] = df["text"].fillna("").astype(str)
df["label_id"] = df["label_id"].astype(int)
df["label_name"] = df["label_name"].fillna("unknown").astype(str)
df = df[df["text"].str.strip().astype(bool)].reset_index(drop=True)

id_to_name = df.drop_duplicates("label_id").set_index("label_id")["label_name"].to_dict()
id_to_name


{0: 'разговорный стиль', 1: 'официально-деловой стиль'}

In [30]:
# Баланс классов
label_counts = df["label_id"].value_counts().sort_index()
label_counts.to_frame("count")


,count
label_id,
0,8000
1,8000


In [31]:
# Формируем трейн/валидацию
train_df, valid_df = train_test_split(
    df[["text", "label_id"]],
    test_size=VAL_SIZE,
    random_state=RANDOM_SEED,
    stratify=df["label_id"],
)
train_df.shape, valid_df.shape


((12800, 2), (3200, 2))

In [32]:
# Подготавливаем временные файлы формата fastText
def format_label(label_id: int) -> str:
    return f"__label__{label_id}"

def export_fasttext_file(frame, prefix: str) -> Path:
    tmp = NamedTemporaryFile("w", encoding="utf-8", delete=False, suffix=f"_{prefix}.txt")
    for text, label_id in zip(frame["text"], frame["label_id"]):
        normalized = " ".join(text.split())
        tmp.write(f"{format_label(int(label_id))} {normalized}\n")
    tmp_path = Path(tmp.name)
    tmp.close()
    return tmp_path

train_file = export_fasttext_file(train_df, "train")
valid_file = export_fasttext_file(valid_df, "valid")
train_file, valid_file


(PosixPath('/tmp/tmp41u9sfwj_train.txt'),
 PosixPath('/tmp/tmp_ywhybvq_valid.txt'))

In [33]:
# Обучаем fastText
model = fasttext.train_supervised(
    input=train_file.as_posix(),
    epoch=30,
    lr=0.5,
    wordNgrams=3,
    dim=200,
    loss="ova", #softmax
    verbose=2,
)
model


Read 0M words
Number of words:  96782
Number of labels: 2
Progress: 100.0% words/sec/thread:  613874 lr:  0.000000 avg.loss:  0.025283 ETA:   0h 0m 0s


In [34]:
# Оцениваем качество на валидации
sorted_ids = sorted(id_to_name)
target_names = [id_to_name[idx] for idx in sorted_ids]

def predict_ids(texts):
    labels, probs = model.predict(texts)
    label_ids = [int(label[0].replace("__label__", "")) for label in labels]
    confidences = [prob[0] for prob in probs]
    return label_ids, confidences

y_true = valid_df["label_id"].to_numpy()
y_pred, y_scores = predict_ids(valid_df["text"].tolist())

val_accuracy = accuracy_score(y_true, y_pred)
val_report_text = classification_report(
    y_true, y_pred, labels=sorted_ids, target_names=target_names, digits=3
)
val_report_dict = classification_report(
    y_true, y_pred, labels=sorted_ids, target_names=target_names, output_dict=True
)

print(f"Accuracy: {val_accuracy:.3f}")
print(val_report_text)


Accuracy: 0.964
                          precision    recall  f1-score   support

       разговорный стиль      0.967     0.961     0.964      1600
официально-деловой стиль      0.961     0.968     0.964      1600

                accuracy                          0.964      3200
               macro avg      0.964     0.964     0.964      3200
            weighted avg      0.964     0.964     0.964      3200



In [35]:
# Сравниваем предсказания модели с hint по всему корпусу
comparison_df = df[["text", "label_id"]].copy()
comparison_df["hint_label_name"] = comparison_df["label_id"].map(id_to_name)

pred_ids, pred_probs = predict_ids(comparison_df["text"].tolist())
comparison_df["pred_label_id"] = pred_ids
comparison_df["pred_label_name"] = comparison_df["pred_label_id"].map(id_to_name)
comparison_df["pred_confidence"] = pred_probs

mismatch_mask = comparison_df["pred_label_id"] != comparison_df["label_id"]
mismatch_rate = mismatch_mask.mean()
print(f"Доля расхождений с hint: {mismatch_rate:.2%} ({mismatch_mask.sum()} из {len(comparison_df)})")
pd.crosstab(comparison_df["hint_label_name"], comparison_df["pred_label_name"])


Доля расхождений с hint: 0.71% (114 из 16000)


pred_label_name,официально-деловой стиль,разговорный стиль
hint_label_name,,
официально-деловой стиль,7948,52
разговорный стиль,62,7938


In [36]:
# Примеры расхождений между hint и предсказаниями
comparison_df.loc[mismatch_mask, ["text", "hint_label_name", "pred_label_name", "pred_confidence"]].head(10)


,text,hint_label_name,pred_label_name,pred_confidence
216,"Здравствуйте. Я очень рад, что Вы ,,закорешили...",разговорный стиль,официально-деловой стиль,0.798197
252,Для первой это второй в жизни финал турнира Бо...,официально-деловой стиль,разговорный стиль,0.914911
346,есть собака или кот - приноси в лдпр! нафигачи...,разговорный стиль,официально-деловой стиль,0.766304
407,"Макс, Япончик - Япончик (наст. имя - Мойше-Яко...",разговорный стиль,официально-деловой стиль,0.995105
430,"Мустафа Джемилев несколько ошибся, когда 2 янв...",разговорный стиль,официально-деловой стиль,0.967909
546,"Сообщалось, что один из приятелей Лебедева ока...",официально-деловой стиль,разговорный стиль,0.515630
570,"Степан, шоб она текла до утра!Вы ничего не пон...",разговорный стиль,официально-деловой стиль,0.901931
715,Нам остается только принять это как свершивший...,официально-деловой стиль,разговорный стиль,0.985053
1006,"Опять ни слова о том, что буржуйское ЕдРо преп...",разговорный стиль,официально-деловой стиль,0.637041
1251,"Владимир, относиться к жирику серьезно, и прин...",разговорный стиль,официально-деловой стиль,0.622469


In [37]:
# Пример инференса
sample_text = "Вышлем финансовый отчет и согласуем детали контракта в ближайшие два дня."
pred_label, pred_prob = model.predict(sample_text, k=1)
pred_id = int(pred_label[0].replace("__label__", ""))
id_to_name[pred_id], pred_prob[0]


('официально-деловой стиль', np.float64(0.9855064153671265))

In [38]:
# Сохраняем модель и очищаем временные файлы
model_path = MODEL_DIR / "fasttext_style.bin"
model.save_model(model_path.as_posix())

for tmp_path in (train_file, valid_file):
    if tmp_path.exists():
        tmp_path.unlink(missing_ok=True)

model_path


PosixPath('models/fasttext_style.bin')

In [39]:
# Сводная аналитика fastText baseline
print("=== Результаты fastText baseline ===")
macro_f1 = val_report_dict.get("macro avg", {}).get("f1-score", float("nan"))
weighted_f1 = val_report_dict.get("weighted avg", {}).get("f1-score", float("nan"))
print(f"Валидация: accuracy = {val_accuracy:.3f}, macro F1 = {macro_f1:.3f}, weighted F1 = {weighted_f1:.3f}")

print("=== Расхождения с исходными метками ===")
mismatch_count = int(mismatch_mask.sum())
total_count = int(len(comparison_df))
print(f"Доля несоответствий с hint: {mismatch_rate:.2%} ({mismatch_count} из {total_count})")

print("=== Корпус и сплиты ===")
print(f"Всего сегментов: {len(df)}")
print(f"Обучающая выборка fastText: {len(train_df)} · валидация: {len(valid_df)}")
print("Баланс классов:")
for label_id, count in label_counts.sort_index().items():
    label_name = id_to_name.get(label_id, str(label_id))
    print(f"  {label_name} ({label_id}): {int(count)}")

print("=== Артефакты ===")
print(f"Модель fastText сохранена в: {model_path}")
print("Временные файлы fastText очищены после обучения.")



=== Результаты fastText baseline ===
Валидация: accuracy = 0.964, macro F1 = 0.964, weighted F1 = 0.964
=== Расхождения с исходными метками ===
Доля несоответствий с hint: 0.71% (114 из 16000)
=== Корпус и сплиты ===
Всего сегментов: 16000
Обучающая выборка fastText: 12800 · валидация: 3200
Баланс классов:
  разговорный стиль (0): 8000
  официально-деловой стиль (1): 8000
=== Артефакты ===
Модель fastText сохранена в: models/fasttext_style.bin
Временные файлы fastText очищены после обучения.
